In [6]:
import sys
import os
import pandas as pd
import numpy as np

def script_type():
    return "column"


def custom_exec(org_data,currentTitleIndex, data_interface):
    print(currentTitleIndex)
    #这里填写您的代码 org_data 是dataframe 待处理数据

    #处理完之后返回一个dataframe
    return org_data.iloc[:,currentTitleIndex]

df = pd.read_csv('C:\\Users\\ThinkPad\\Desktop\\20200320_163026\\data.csv')
titleindex = 2
df = custom_exec(df, titleindex, "")
print(df)

2
0      2008-03-05
1      2008-07-30
2      2008-07-30
3      2008-07-31
4      2008-07-31
5      2008-08-04
6      2008-08-04
7      2008-08-04
8      2008-08-04
9      2008-08-04
10     2008-08-04
11     2008-08-04
12     2008-08-04
13     2008-08-04
14     2008-08-04
15     2008-08-04
16     2008-08-04
17     2008-08-04
18     2008-08-04
19     2008-08-04
20     2008-08-04
21     2008-08-04
22     2008-08-04
23     2008-08-04
24     2008-08-04
25     2008-08-04
26     2008-08-04
27     2008-08-04
28     2008-08-04
29     2008-08-04
          ...    
170    2008-08-04
171    2008-08-04
172    2008-08-04
173    2008-08-04
174    2008-08-04
175    2008-08-04
176    2008-08-04
177    2008-08-04
178    2008-08-05
179    2008-08-05
180    2008-08-05
181    2008-08-05
182    2008-08-05
183    2008-08-05
184    2008-08-05
185    2008-08-05
186    2008-08-05
187    2008-08-05
188    2008-08-05
189    2008-08-05
190    2008-08-05
191    2008-08-05
192    2008-08-05
193    2008-08-05
194    2

In [1]:
import pandas as pd
Efile =  pd.ExcelFile('G:\\NT数据\\1\\爱农\\上海郝魏（开）\\上海郝魏商贸有限公司.xlsx')
print(type(Efile))

<class 'pandas.io.excel._base.ExcelFile'>


In [6]:
import pandas as pd
import numpy as np
import re
import datetime
import os
output_path = 'I:\\南通数据\\'
if not os.path.exists(output_path + './资金流水'):
    os.mkdir(output_path + './资金流水')


all_流水 = pd.read_csv('G:\流水完整.csv',encoding = 'utf-8',dtype = object)
all_流水 = all_流水.applymap(lambda x: re.sub('nan','',str(x)))
all_流水['交易金额'] = all_流水['交易金额'].map(lambda x: float(x))  #交易金额指定浮点型
all_流水['交易余额'] = all_流水['交易余额'].map(lambda x: np.NaN if str(x)=='' else x)
all_流水['交易余额'] = all_流水['交易余额'].map(lambda x: float(x))
all_流水['交易时间'] = all_流水['交易时间'].map(lambda x:str(datetime.datetime.strptime(x, '%Y-%m-%d%H:%M:%S')) if len(x)== 18 else x)


all_账户 = pd.read_excel('G:\账户合并.xlsx',encoding = 'gbk',dtype = object)
df_账户 = all_账户[all_账户['账户开户名称'] != '']
df_账户['交易卡号'] = df_账户['交易卡号'].str.split('_', 1,expand = True).iloc[:,0]   # 账户开户信息中卡号去掉_后缀
df_账户['交易卡号'] = df_账户['交易卡号'].str.split('-', 1,expand = True).iloc[:,0]  # 账户开户信息中卡号去掉-后缀
df_账户['开户网点'][df_账户['开户网点']==''] = df_账户['账号开户银行'][df_账户['开户网点']=='']  # 网点为空用账户开户银行填充


all_证件 = pd.read_excel('G:\证件合并.xlsx',encoding = 'gbk',dtype = object)

all_商户 = pd.read_csv('G:\商户合并.csv',encoding = 'utf-8',dtype = 'object')
all_商户['创建时间']=all_商户['创建时间'].map(lambda x:str(datetime.datetime.strptime(x,'%Y%m%d%H%M%S')) if len(x)== 14 else x)






#--------------------------------------------------保留7月以后---------------------------------------------------------#
# df = df[(df['交易时间'].str[0:4].isin(['2019']) ) & (df['交易时间'].str[5:7].isin(['07','08','09','10','11']) )]

df = all_流水[(all_流水['交易时间'].str[0:4].isin(['2019','2020']) ) ]


#--------------------------------------------------保留7月以后---------------------------------------------------------#


df_S1 = df.groupby('交易卡号').size().reset_index()
df_S2 = pd.DataFrame(columns = ['交易卡号','Size','False','True'])
df_S3 = pd.DataFrame(columns = ['交易卡号','交易户名','证件号码','夜间交易次数','最早时间','最晚时间','余额','总和','净和','入向','出向','入向对手数','出向对手数','收付检验起始值','Path'])
for i in df_S1.index:
    df_every = df.groupby('交易卡号').get_group(df_S1['交易卡号'][i])
    bool_S = df_every.duplicated(subset = ['交易卡号','交易时间','交易金额','交易余额','收付标志','交易对手账卡号','交易币种'],keep = 'last').value_counts()
    df_S2.loc[i] = {'交易卡号':df_every['交易卡号'].iloc[0],
                        'Size':df_every.shape[0],
                        'False':bool_S[0],
                        'True':bool_S.map(lambda x:0 if len(bool_S)==1 else bool_S[1]).values[0]
                    }
    df_every = df_every.drop_duplicates(subset = ['交易卡号','交易时间','交易金额','交易余额','收付标志','交易对手账卡号','交易币种'],keep = 'last')
    df_every.to_excel(output_path+"资金流水\\" + str(df_every['交易户名'].iloc[0])+str(df_every['交易卡号'].iloc[0])+".xlsx",index = None)  # 拆表
    df_night = df_every[df_every['交易时间'].str[11:13].isin(['00','01','02','03','04','05','06'])]
    df_S3.loc[i] = {'交易卡号':df_every['交易卡号'].iloc[0],
                    '交易户名':df_every['交易户名'].iloc[0],
                    '证件号码':df_every['交易证件号码'].iloc[0],
                    '夜间交易次数':len(df_night),
                    '最早时间':df_every['交易时间'].min(),
                    '最晚时间':df_every['交易时间'].max(),
                    '余额':df_every['交易余额'].loc[df_every.index.max()],
                    '总和':df_every['交易金额'].sum(),
                    '净和':format(df_every['交易金额'][df_every['收付标志']=='进'].sum()-df_every['交易金额'][df_every['收付标志']=='出'].sum(), '.2f'),
                    '入向':df_every['交易金额'][df_every['收付标志']=='进'].sum(),
                    '出向':df_every['交易金额'][df_every['收付标志']=='出'].sum(),
                    '入向对手数':len(df_every['交易对手账卡号'][df_every['收付标志']=='进'].drop_duplicates()),
                    '出向对手数':len(df_every['交易对手账卡号'][df_every['收付标志']=='出'].drop_duplicates()),
                    '收付检验起始值':df_every['交易余额'].iloc[0]+df_every.apply(lambda x: x['交易金额'] if x['收付标志']=='出' else -x['交易金额'],axis=1).iloc[0],
                    'Path':'\n'.join(df_every['path'].drop_duplicates().tolist())
                   }
    print(i,end = ' ')
df_S = pd.merge(df_S3,df_S2,how = 'left',on = '交易卡号')

dfs_账户银行 = pd.merge(df_S,df_账户.drop_duplicates(subset='交易卡号'),how = 'left',on = '交易卡号')
df_S['开户网点'] = dfs_账户银行['开户网点']
df_S['账户状态'] = dfs_账户银行['账户状态']
df_S.to_excel(output_path + "交易统计.xlsx",index = None)
# 统计完了，按照六大要素+币种 删除重复项（因为手动改了-101与-102的双向交易的卡号后缀，-101出-102进这类记录会造成自实体链接，不影响）
all_流水完整 = df.drop_duplicates(subset = ['交易卡号','交易时间','交易金额','交易余额','收付标志','交易对手账卡号','交易币种'],keep = 'last')
# all_流水完整.to_csv(output_path + "流水完整.csv",index = None)
# print(all_流水完整.shape)

df_流水截取 = all_流水完整.iloc[:,:32]
df_流水截取.to_csv(output_path + "流水截取.csv",index = None)
print(df_流水截取.shape+tuple(df_流水截取.columns))

df_账户截取 = df_账户[df_账户['交易卡号'].isin(all_流水完整['交易卡号'])].iloc[:,:43]
df_账户截取.to_excel(output_path + "账户截取.xlsx",index = None)
print(all_账户.shape+tuple(all_账户.columns))

df_证件截取 = all_证件[all_证件['证照号码'].isin(all_流水完整['交易证件号码'])].iloc[:,:22]
df_证件截取.to_excel(output_path + "证件截取.xlsx",index = None)
print(all_证件.shape+tuple(all_证件.columns))

df_商户截取 = all_商户[(all_商户['订单日期'].str[0:4].isin(['2019']) ) & (all_商户['账户号'].isin(all_流水完整['交易对手账卡号']) )].iloc[:,:17]
df_商户截取.to_csv(output_path + "商户截取.csv",index = None)
print(df_商户截取.shape+tuple(all_商户.columns))


# all_账户.iloc[:,:43].to_excel(output_path + "账户合并.xlsx",index = None)
# all_证件.iloc[:,:22].to_excel(output_path + "证件合并.xlsx",index = None)
# all_商户.iloc[:,:17].to_csv(output_path + "商户合并.csv",index = None)
# all_流水完整.iloc[:,:32].to_csv(output_path + "流水完整.csv",index = None)


0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

In [4]:
all_商户['创建时间'].min()

'2018-12-21 12:16:36'